# Streamlit App을 EC2에서 서비스 

1. App을 Github에 올린다.
    - Github에 올릴 내용
    	- App source
    	- requirements.txt: 설치할 lib 목록
	- openai api 키와 같은 비용이 들어가는 설정 정보는 github에 올리지 않는다.
2. [EC2 instance에 인바운드(inbound) 규칙 추가](02_EC2생성.ipynb#보안그룹의-인바운드(Inbound)-규칙-추가하기)
    - streamlit을 서비스할 port를 열어준다.(EC2 instance 생성 할때 설정하지 않은 경우)
        - streamlit 기본 포트: 8501
    - streamlit 실행시 포트 변경
        - `streamlit run app.py --server.port <포트번호>`
        - 만약 포트번호를 지정해서 실행할 경우 그 포트번호로 **인바운드 규칙**을 설정한다.
3. Streamlit 실행
    - `nohup streamlait run 실행파일.py & `

# Django Project를 EC2에서 서비스

## nginx 설치
- Nginx는 오픈소스 웹서버 프로그램이다.
- Django 웹 애플리케이션의 성능, 안정성, 보안을 위해 nginx와 연동해 서비스를 한다.
- Nginx는 http 요청을 받는 것, http 응답을 생성하는 역할과 static 파일 서비스를 담당한다.
- Django Application(View) 요청에 대한 처리는 WSGI와 통신해서 실행한다.
  
    ```bash
    sudo apt update
    sudo apt upgrade
    sudo apt install nginx
    ```

## Gunicorn 설치 및 설정

- Gunicorn
    - WSGI(Web Server Gateway Interface) 서버
      - 파이썬 웹 어플리케이션 실행환경
- **WSGI의 작동 방식**
    - 클라이언트가 HTTP 요청을 보낸다.
    - 웹 서버(Nginx, Apache 등)가 요청을 받아 WSGI 서버(Gunicorn)에 전달한다.
      - static 요청은 웹서버가 처리하고 동적 요청(django view)는 WSGI 서버가 실행한다.
    - WSGI 서버는 요청을 파이썬 웹 애플리케이션(Django, Flask 등)에 전달하여 실행한다.
    - 파이썬 웹 애플리케이션은 요청을 처리하고 응답을 생성하여 WSGI 서버로 반환한다.
    - WSGI 서버는 응답을 웹 서버로 전달하고, 웹 서버는 Http응답정보를 클라이언트에 반환한다.

![img](figures/python/nginx_gunicorn.png)

- **Gunicorn 설치**
  -  django project 에서 가상 환경 설치 후 gunicorn 서버 설치한다.
    ```bash
    uv venv .venv --python=3.12
    source .venv/bin/activate   # 가상환경 활성화 
    # django 등 필요 패키지 설치한다.
    uv pip install gunicorn
    ```

## Django 프로젝트 설정 (중요)
`/home/ubuntu/mypoll/config/settings.py` 수정:
- **ALLOWED_HOSTS** 설정
    - 서비스할 host를 설정. 어떤 도메인/IP 주소로 들어온 요청만 처리할 것인지 설정한다.
    - "EC2 instance의 public ip 주소" 를 설정한다.
    - '*'
- `SECRET_KEY` 를 보안적 이유로 외부에 저장하고 호출 하도록 하는 것이 좋다.
- `DEBUG=False` 설정
   - 운영시 디버그 모드를 False로 변경한다.
     - 실행 중 오류 메세지가 사용자에게 출력 되지 않는다.
     - 코드 변경시 자동 적용 되지 않는다.
 - 예)
    ```python
    DEBUG = False
    ALLOWED_HOSTS = ['*']
    SECRET_KEY = os.getenv('DJANGO_SECRET_KEY')
    ```

- **STATIC 및 MEDIA 파일 설정**
    ```python
    STATIC_ROOT = '/var/www/<<project_name>>/static'   # collectstatic 실행시 static 파일들을 모을 디렉토리. 웹서버 연동시 필요하다.
    STATIC_URL = '/static/'                            # static 파일 요청시 시작 url
    MEDIA_ROOT = '/var/www/<<project_name>>/media'                    # 업로드 파일 저장 디렉토리
    MEDIA_URL = '/media/'
    ```
- **STATIC_ROOT, MEDIA_ROOT** 디렉토리 생성 및 소유권 변경
  ```bash
    sudo mkdir -p /var/www/<<project_name>>/static             # -p 부모 디렉토리까지 중간 경로가 없으면 모두 자동으로 생성
    sudo chown $USER:$USER /var/www/<<project_name>>/static    # chown user:userGroup 대상파일/디렉토리: 파일이나 디렉토리의 소유자(owner)와 그룹(group)을 변경하는 명령어. 
                                                               # $USER : 현재 로그인한 사용자명을 가지는 환경변수
    sudo chmod 755 sudo mkdir -p /var/www/<<project_name>>/static

    sudo mkdir -p /var/www/<<project_name>>/media
    sudo chown $USER:$USER /var/www/<<project_name>>/media
    sudo chmod 755 sudo mkdir -p /var/www/<<project_name>>/media
  ```

- **collectstatic** 실행
    ```bash
    python manage.py collectstatic
    ```
- DB 생성 관련 설정
    ```bash
    python manage.py makemigrations polls account chat
    python manage.py migrate
    ```

- 개발서버 실행 후 확인
    - `python manage.py runserver 0.0.0.0:8000`

> - **chown**
>   - 파일이나 디렉토리의 소유자와 그룹을 변경하는 리눅스 명령어
>   - `chown user file`, `chown user:group file`, 
>       - 옵션: `-R` - 재귀적으로 하위 디렉토리,파일까지 재귀적으로 모두 적용
> 
> - **chmod**
>    - 파일이나 디렉토리의 권한(permission) 을 변경하는 리눅스 명령어
>    - `chmod 권한 파일`
>       - 옵션: `-R` - 재귀적으로 하위 디렉토리,파일까지 재귀적으로 모두 적용
>    - 권한 숫자
>       - r(읽기): 4, w(쓰기): 2, x(실행): 1
>       - 권한 숫자의 합계로 **권한지정**
>           - 7: rwx, 5: r-x, 6: rw-
>           - 소유자, 소유자그룹, 기타 옵션을 권한 숫자합계 3개로 지정한다.
>           - 예) 755 (소유자: rwx / 그룹: r-x / 기타: r-x)

## Gunicorn 실행 (테스트)
```bash
cd /home/ubuntu/<<project_name>>

gunicorn --bind 0.0.0.0:8000 config.wsgi:application
```

- 웹 브라우저에서 `http://<<your-ec2-public-ip>>:8000`로 접속하여 확인.

## Gunicorn 서비스 생성 (컴퓨터 시작시 자동 실행 하도록 설정)
- **서비스(데몬-daemon)** 이란
  - 서비스(service)는 운영체제가 부팅 시 또는 필요할 때 자동으로 실행·관리하는 상주 프로세스(daemon)이다.
  - 리눅스에서는 `systemd` 가 관리하며 `systemctl` 을 이용해 관리한다. (시작, 재시작, 종료)

### gunicorn을 서비스로 등록
- `/etc/systemd/system/` 아래 서비스 실행 설정파일 생성

```bash
sudo nano /etc/systemd/system/mypoll_chat-wsgi.service
# 아래 항목 작성 후 저장(control+s) 종료(control+x)
```
- 다음 항목 입력
  
```ini
[Unit]
Description=Gunicorn daemon for Django Project
After=network.target                 # 네트워크가 활성화된 후 시작

[Service]
User=ubuntu                          # 서비스 실행 사용자
Group=ubuntu                         # 서비스 실행 사용자 그룹

Environment="PATH=/home/ubuntu/<<Django Project 디렉토리>>/.venv/bin"
Environment="DJANGO_SETTINGS_MODULE=config.settings"

WorkingDirectory=/home/ubuntu/<<Django Project 디렉토리>> # 작업 디렉토리
ExecStart=/home/ubuntu/<<Django Project 디렉토리>>/.venv/bin/gunicorn --workers 3 --bind unix:/run/mypoll_chat/gunicorn.sock config.wsgi:application  # gunicorn 실행명령어

[Install]
WantedBy=multi-user.target      # 다중 사용자 모드에서 시작

# /run/mypoll_chat 디렉토리가 서비스 실행시 생성되도록 설정(/run 은 휘발성이어서 종료시 디렉토리가 삭제될 수 있다.)
[Service]
RuntimeDirectory=mypoll_chat
RuntimeDirectoryMode=0755
```
- socket 파일을 저장 디렉토리 생성 및 설정 (/run/mypoll_chat)
    ```bash
     sudo mkdir -p /run/mypoll_chat
     sudo chown $USER:www-data /run/mypoll_chat
     sudo chmod 775 /run/mypoll_chat
    ```

- 서비스 시작 (mypoll_chat-wsgi.service 를 서비스로 시작 및 등록)
  ```bash
  sudo systemctl start mypoll_chat-wsgi      # 서비스로 시작
  sudo systemctl enable mypoll_chat-wsgi     # 자동등록
  sudo systemctl status mypoll_chat-wsgi     # 상태확인
  ```

> - linux 에서 `/run`과 `/var`
>     - `/run`: 부팅시 생성되는 런터임 저장소. 실행중에만 사용하는 파일들을 저장할 디렉토리
>     - `/var`: 영구 저장장 파일들 저장할 디렉토리로 서비스가 지속적으로 사용할 파일들을 저장한다.

## Nginx

### 설치
```bash
sudo apt update
sudo apt upgrade
sudo apt install nginx
```
- 설치 후 nginx 는 서비스로 등록된다.
- http://<<AWS public ip>> 확인

### Django Project - ngix 연동 설정

- site 설정파일 생성

```bash
sudo nano /etc/nginx/sites-available/<<project이름>>
```

- 다음 설정을 추가:
    - 주의: `location /static/`, `location /media/` 경로 설정시 뒤에 `/` 생략 하면 안됨
```nginx
server {
    listen 80;
    server_name <<ec2-public-ip>>;

    location /static/ {
        alias <<STATIC_ROOT 디렉토리>>;     
    }

    location /media/ {
        alias <<MEDIA_ROOT 디렉토리>>;                
    }

    location / {
        include proxy_params;
        proxy_pass http://unix:<<gunicorn socket파일 경로>>;
    }
}
```


심볼릭 링크 생성 및 Nginx 재시작:
```bash
sudo ln -s /etc/nginx/sites-available/<<project이름>> /etc/nginx/sites-enabled
sudo nginx -t    # 설정파일 검증
sudo systemctl restart nginx # nginx 서비스 재시작
sudo systemctl status nginx  # 서비스 실행 상태 확인
```

## 코드/설정 변경시
- django 코드 변경시 `gunicorn`, `nginx` 재시작 한다.
```bash
sudo systemctl daemon-reload          # 설정파일 변경시 실행.
sudo systemctl restart gunicorn서비스
sudo systemctl restart nginx
```

## 서버 상태 확인 및 로그 모니터링
- **서비스 상태 확인**
```bash
sudo systemctl status gunicorn서비스
sudo systemctl status nginx
```
- **로그 확인**
```bash
tail /var/log/nginx/error.log
```

# Django Application - RDS DB 연결

1. [RDS에 MySQL DB 설치](04_RDS.ipynb#데이터베이스-생성-설정)
2. `pymysql` 설치
```bash
conda activate django
pip install pymysql
```

3. DB 연결 후 database 생성
   - `mysql -u 계정 -p -h 앤드 포인트`
   - 연결 후
       - Database 확인: `show databases`
       - Database 생성: `create database mypoll`

4. **settings.py** 설정
    
    - **mysql lib 설정**
        ```python
        import pymysql
        pymysql.install_as_MySQLdb()
        ```
    
    - **Database 연결 설정***
        ```python
        DATABASES = {
            'sqlite3': {
                'ENGINE': 'django.db.backends.sqlite3',
                'NAME': BASE_DIR / 'db.sqlite3',
            },
            'default': {
                'ENGINE': 'django.db.backends.mysql',
                'NAME': '<<Database 이름>>',
                'USER': '<<username>>',
                'PASSWORD': '<<password>>',
                'HOST': '<<RDS DB instance end point>>',
                'PORT': '3306',
            }
        }
        ```

5. 서비스 reload
```bash
sudo systemctl daemon-reload     
sudo systemctl restart gunicorn서비스
sudo systemctl restart nginx
```